In [19]:
import pandas as pd
import numpy as np

# Dataframe

In [20]:
df=pd.read_csv("C:\\Users\\Saravanan\\OneDrive\\Desktop\\Datasets_data\\Student_Performance.csv")

In [21]:
df

,Hours Studied,Previous Scores,Extracurricular Activities,Sleep Hours,Sample Question Papers Practiced,Performance Index
0,7,99,Yes,9,1,91.0
1,4,82,No,4,2,65.0
2,8,51,Yes,7,2,45.0
3,5,52,Yes,5,2,36.0
4,7,75,No,8,5,66.0
...,...,...,...,...,...,...
9995,1,49,Yes,4,2,23.0
9996,7,64,Yes,8,5,58.0
9997,6,83,Yes,8,5,74.0
9998,9,97,Yes,7,0,95.0


# Observation

# Unsupervised


In [4]:
df.columns

Index(['Hours Studied', 'Previous Scores', 'Extracurricular Activities',
       'Sleep Hours', 'Sample Question Papers Practiced', 'Performance Index'],
      dtype='object')

In [5]:
for column in df.columns:
  print(column," : ",df[column].unique())
  print("_________")
     

Hours Studied  :  [7 4 8 5 3 6 2 1 9]
_________
Previous Scores  :  [99 82 51 52 75 78 73 45 77 89 91 79 47 72 83 54 96 74 85 61 62 84 94 90
 44 70 67 97 59 55 68 63 46 76 43 81 93 98 48 92 64 88 60 87 40 69 80 95
 49 50 53 71 56 58 66 65 57 41 42 86]
_________
Extracurricular Activities  :  ['Yes' 'No']
_________
Sleep Hours  :  [9 4 7 5 8 6]
_________
Sample Question Papers Practiced  :  [1 2 5 6 0 8 3 4 9 7]
_________
Performance Index  :  [ 91.  65.  45.  36.  66.  61.  63.  42.  69.  84.  73.  27.  33.  68.
  43.  67.  70.  30.  71.  85.  57.  35.  49.  83.  74.  39.  58.  47.
  60.  32.  64.  54.  17.  53.  75.  52.  78.  38.  98.  87.  41.  81.
  15.  88.  95.  29.  21.  76.  25.  34.  50.  56.  82.  23.  46.  92.
  77.  86.  44.  94.  40. 100.  31.  26.  18.  51.  72.  16.  28.  89.
  48.  37.  62.  59.  19.  79.  22.  10.  90.  80.  24.  20.  96.  55.
  97.  12.  93.  14.  99.  11.  13.]
_________


In [13]:


continuous_columns = ['Previous Scores', 'Sample Question Papers Practiced', 'Performance Index']
category_columns = ['Hours Studied', 'Extracurricular Activities', 'Sleep Hours']



# Data Cleaning

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 6 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Hours Studied                     10000 non-null  int64  
 1   Previous Scores                   10000 non-null  int64  
 2   Extracurricular Activities        10000 non-null  object 
 3   Sleep Hours                       10000 non-null  int64  
 4   Sample Question Papers Practiced  10000 non-null  int64  
 5   Performance Index                 10000 non-null  float64
dtypes: float64(1), int64(4), object(1)
memory usage: 468.9+ KB


In [8]:
df.isnull().sum()

Hours Studied                       0
Previous Scores                     0
Extracurricular Activities          0
Sleep Hours                         0
Sample Question Papers Practiced    0
Performance Index                   0
dtype: int64

In [9]:
df.duplicated().sum()

127

In [10]:
df.drop_duplicates(inplace=True)

In [11]:
df.duplicated().sum()

0

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9873 entries, 0 to 9999
Data columns (total 6 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Hours Studied                     9873 non-null   int64  
 1   Previous Scores                   9873 non-null   int64  
 2   Extracurricular Activities        9873 non-null   object 
 3   Sleep Hours                       9873 non-null   int64  
 4   Sample Question Papers Practiced  9873 non-null   int64  
 5   Performance Index                 9873 non-null   float64
dtypes: float64(1), int64(4), object(1)
memory usage: 539.9+ KB


# Hypothesis testing / Statistical analysis

Continuous - Central Limit Theorem, 1-Tailed Test (one-sample t-test)

Continuous vs. Continuous - Correlation, 2-Tailed Test (two-sample t-test)

Continuous vs. Categorical - ANOVA (Analysis of Variance)

Categorical vs. Categorical - Chi-Square Test

In [22]:
continuous_columns = ['Previous Scores', 'Sample Question Papers Practiced', 'Performance Index']
category_columns = ['Hours Studied', 'Extracurricular Activities', 'Sleep Hours']


In [32]:
def hypothesis_testing(df, continuous_columns=[], category_columns=[], sampling_rate=0.20, sampling_frac=0.05):
    # central limit theorem
    import pandas as pd
    OneContinuousColumn_result = {}
    hypothesis_df = pd.DataFrame(columns=df.columns, index=df.columns)
    if continuous_columns:
        sample_size = int(sampling_rate * len(df[continuous_columns[0]]))
        for column in continuous_columns:
            population = df[column].values
            population_mean = population.mean()
            sample_mean = []
            for i in range(40):
                sample = np.random.choice(population, sample_size)
                sample_mean.append(sample.mean())
            OneContinuousColumn_result[column] = {
                "Population Mean": population_mean,
                "Sample Mean": np.mean(sample_mean)
            }
    return OneContinuousColumn_result

In [33]:
ContinuousColumn_result = hypothesis_testing(df, continuous_columns, category_columns)

In [34]:
ContinesColumn_result_df = pd.DataFrame(ContinuousColumn_result)
ContinesColumn_result_df

,Previous Scores,Sample Question Papers Practiced,Performance Index
Population Mean,69.445700,4.583300,55.2248
Sample Mean,69.464012,4.579375,55.2113
